In [4]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Output')
cur_date = "041322"

library(readxl)
library(tidyverse)
library(caret)
# library(e1071)
# library(Hmisc)
# library(glmnet)
library(pROC)
library(randomForest)

#reading in file
molecular_feature_df = read_csv("20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)
demographics_data = data.frame(read_excel("SubjectInfo_022822.xlsx", sheet = 2))

Rows: 59 Columns: 3873
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr    (1): sample
dbl (3872): MF_2, MF_3, MF_4, MF_5, MF_6, MF_7, MF_11, MF_12, MF_13, MF_14, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
head(molecular_feature_df)
head(demographics_data)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


,Original_SubjectID,SubjectID,Exposure,Sex,Race,Age,BMI
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,NBL214,Ecig_214,Ecig,F,O,19,20.9
2,NBL221,Ecig_221,Ecig,M,As,21,24.0
3,NBL222,Ecig_222,Ecig,F,W,20,30.2
4,NBL223,Ecig_223,Ecig,F,B,19,19.9
5,NBL229,Ecig_229,Ecig,F,W,39,35.1
6,NBL230,Ecig_230,Ecig,F,W,40,45.6


Running random forest models including sex as another predictor. 

In [6]:
# creating 1 df
demographics_data = demographics_data %>%
    select(c("SubjectID", "Sex")) %>%
    rename(Smoking_Status = SubjectID)
           
mf_df = full_join(demographics_data, molecular_feature_df) %>%
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"),
          Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0"))

mf_df_wo_nicotine = mf_df %>%
    #removing nicotine and cotinine
    select(-c(MF_97, MF_5))

head(mf_df)

Joining, by = "Smoking_Status"


,Smoking_Status,Sex,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,20.95330,21.69571,20.65056,21.89841,17.48492,20.32206,19.86974,20.76839,⋯,19.28804,19.28720,19.50373,18.00832,18.27601,17.03340,18.93808,18.98799,18.67967,18.27810
2,1,1,22.40718,24.32335,22.70868,21.28516,16.42013,22.63653,15.58863,22.70086,⋯,16.91953,19.17695,18.31817,17.24447,14.91195,19.04946,18.60969,19.40770,19.44686,19.20890
3,1,0,22.45404,24.87327,23.44948,27.79174,17.51114,22.93427,22.70884,22.74215,⋯,16.74891,19.33585,17.32943,19.62663,17.29552,19.11852,19.29798,16.80878,19.39754,19.30157
4,1,0,22.92114,24.83565,23.21817,21.58997,17.88701,23.46244,22.49399,23.15348,⋯,16.74502,19.10796,18.55441,18.27226,18.45050,19.35907,19.55371,16.48129,19.43559,18.76511
5,1,0,21.50164,22.86175,21.55396,26.46040,18.77247,21.33890,21.03399,20.66345,⋯,19.04752,19.12115,17.31393,18.05895,18.29395,14.99174,18.71581,19.21495,17.28354,16.49232
6,1,0,22.01246,24.00816,22.54138,26.95046,18.68449,22.54351,22.18006,22.52930,⋯,18.96422,19.59157,19.97567,19.41389,19.22670,16.27022,19.10242,19.89876,19.17365,18.97580


# Random Forest

- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [7]:
rf_classification = function(dataset, outcome, pred_outcome){
    #setting for reproducibility
    set.seed(12)
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) #number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 #number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        #will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        #finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)], 
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")


        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        #extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
    }
    
    #taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        #sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    #return training set, matrix and variable importance values
    return(list(data_train, metrics, variable_importance_df))

}

#calling fn
rf_values = rf_classification(mf_df, "Smoking_Status", "pred_Smoking_Status")
rf_values_wo_nicotine = rf_classification(mf_df_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [8]:
#viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("Original RF"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"),
                                 rbind(rf_values[[2]], rf_values_wo_nicotine[[2]]))
rf_confusion_matrix

#viewing most significant features
head(rf_values[[3]])
head(rf_values_wo_nicotine[[3]])

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,None,0.9190476,0.8714286,0.9666667,0.9714286,0.9190476
Original RF,Nicotine & Cotinine,0.6809524,0.7619048,0.6000000,0.7033333,0.6809524


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.4421009
MF_5,4.3522880
MF_292,0.3858630
MF_4803,0.2516448
MF_3345,0.2062216
MF_7423,0.2039394


MF,MeanDecreaseGini
<chr>,<dbl>
MF_292,0.6894046
MF_745,0.2635079
MF_7423,0.2409157
MF_7221,0.2405615
MF_17380,0.2174771
MF_3345,0.2173682


In [9]:
#exporting results
write.csv(rf_confusion_matrix, paste0(Output,"/", cur_date, "_Smoking_Status_Prediction_Results_Covariates.csv"), row.names = FALSE)